In [1]:
import nltk
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from nltk.corpus import stopwords
from nltk.util import ngrams
from collections import Counter

In [2]:
# nltk resources download
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Husnain\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Husnain\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Husnain\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [3]:
data=pd.read_csv(r'csvs\Cleaned_constituency_data.csv')

In [4]:
# Step 1: Preprocess Text
stop_words = set(stopwords.words('english'))
stop_words.add('monetary')
stop_words.add('fund')

def preprocess_text(text):
    tokens = nltk.word_tokenize(str(text).lower())  # Tokenization and lowercasing
    tokens = [word for word in tokens if word.isalpha() and word not in stop_words]  # Remove stopwords and non-alphabetic words
    return tokens

# Apply preprocessing to the 'Extracted_Text' column
data['Processed_Text'] = data['Extracted_Text'].apply(preprocess_text)

In [5]:
data

,Year,Title,Extracted_Text,Region/Authority,Constituency,Processed_Text
0,2024,"IMFC Statement by Christine Lagarde, President...",\r\n \r\n INTERNATIONAL MONETARY AND FINANCIA...,European Central Bank,OBS,"[international, financial, committee, fiftieth..."
1,2024,"IMFC Statement by HE Haitham Al Ghais, Secreta...",\r\n \r\n INTERNATIONAL MONETARY AND FINANCIA...,Organization of the Petroleum Exporting Countries,OBS,"[international, financial, committee, fiftieth..."
2,2024,"IMFC Statement by Ayman Al-Sayari, Governor of...",\r\n \r\n INTERNATIONAL MONETARY AND FINANCIA...,Saudi Arabia,SA,"[international, financial, committee, fiftieth..."
3,2024,"IMFC Statement by Antoine Armand, Minister of ...",INTERNATIONAL MONETARY AND FINANCIAL COMMITTE...,France,FF,"[international, financial, committee, f, iftie..."
4,2024,"IMFC Statement by Luis Caputo, Minister of Eco...",\r\n \r\n INTERNATIONAL MONETARY AND FINANCIA...,Argentina,AG,"[international, financial, committee, fiftieth..."
...,...,...,...,...,...,...
559,2004,IMFC Statement by the Honorable Domenico Sinis...,\r\n International Monetary and \r\nFinancial...,Italy,IT,"[international, financial, committee, tenth, m..."
560,2004,"IMFC Statement by the Honorable John W. Snow, ...",\r\n International Monetary and \r\nFinancial...,United States,US,"[international, financial, committee, tenth, m..."
561,2004,IMFC Statement by H.E. Sadakazu Tanigaki Minis...,\r\n International Monetary and \r\nFinancial...,Japan,JA,"[international, financial, committee, tenth, m..."
562,2004,"IMFC Statement By James D. Wolfensohn, Preside...",\r\n International Monetary and \r\nFinancial...,World Bank,OBS,"[international, financial, committee, tenth, m..."


In [6]:
# Step 2: Extract Bigrams and Trigrams
def get_ngrams(tokenized_text, n=2):
    return list(ngrams(tokenized_text, n))

data['Bigrams'] = data['Processed_Text'].apply(lambda x: get_ngrams(x, 2))
data['Trigrams'] = data['Processed_Text'].apply(lambda x: get_ngrams(x, 3))

# Flatten lists and count occurrences
bigrams_flat = Counter([item for sublist in data['Bigrams'] for item in sublist])
trigrams_flat = Counter([item for sublist in data['Trigrams'] for item in sublist])

# Extract unique words, bigrams, trigrams
unique_terms = set([word for doc in data['Processed_Text'] for word in doc])
unique_bigrams = set(bigrams_flat.keys())
unique_trigrams = set(trigrams_flat.keys())

In [7]:
data

,Year,Title,Extracted_Text,Region/Authority,Constituency,Processed_Text,Bigrams,Trigrams
0,2024,"IMFC Statement by Christine Lagarde, President...",\r\n \r\n INTERNATIONAL MONETARY AND FINANCIA...,European Central Bank,OBS,"[international, financial, committee, fiftieth...","[(international, financial), (financial, commi...","[(international, financial, committee), (finan..."
1,2024,"IMFC Statement by HE Haitham Al Ghais, Secreta...",\r\n \r\n INTERNATIONAL MONETARY AND FINANCIA...,Organization of the Petroleum Exporting Countries,OBS,"[international, financial, committee, fiftieth...","[(international, financial), (financial, commi...","[(international, financial, committee), (finan..."
2,2024,"IMFC Statement by Ayman Al-Sayari, Governor of...",\r\n \r\n INTERNATIONAL MONETARY AND FINANCIA...,Saudi Arabia,SA,"[international, financial, committee, fiftieth...","[(international, financial), (financial, commi...","[(international, financial, committee), (finan..."
3,2024,"IMFC Statement by Antoine Armand, Minister of ...",INTERNATIONAL MONETARY AND FINANCIAL COMMITTE...,France,FF,"[international, financial, committee, f, iftie...","[(international, financial), (financial, commi...","[(international, financial, committee), (finan..."
4,2024,"IMFC Statement by Luis Caputo, Minister of Eco...",\r\n \r\n INTERNATIONAL MONETARY AND FINANCIA...,Argentina,AG,"[international, financial, committee, fiftieth...","[(international, financial), (financial, commi...","[(international, financial, committee), (finan..."
...,...,...,...,...,...,...,...,...
559,2004,IMFC Statement by the Honorable Domenico Sinis...,\r\n International Monetary and \r\nFinancial...,Italy,IT,"[international, financial, committee, tenth, m...","[(international, financial), (financial, commi...","[(international, financial, committee), (finan..."
560,2004,"IMFC Statement by the Honorable John W. Snow, ...",\r\n International Monetary and \r\nFinancial...,United States,US,"[international, financial, committee, tenth, m...","[(international, financial), (financial, commi...","[(international, financial, committee), (finan..."
561,2004,IMFC Statement by H.E. Sadakazu Tanigaki Minis...,\r\n International Monetary and \r\nFinancial...,Japan,JA,"[international, financial, committee, tenth, m...","[(international, financial), (financial, commi...","[(international, financial, committee), (finan..."
562,2004,"IMFC Statement By James D. Wolfensohn, Preside...",\r\n International Monetary and \r\nFinancial...,World Bank,OBS,"[international, financial, committee, tenth, m...","[(international, financial), (financial, commi...","[(international, financial, committee), (finan..."


In [8]:
# Step 3: Create Document-Term Matrix (DTM)
data['Extracted_Text'] = data['Extracted_Text'].fillna("")  # Replace NaN with an empty string
vectorizer = CountVectorizer(ngram_range=(1, 3), stop_words='english')
dtm = vectorizer.fit_transform(data['Extracted_Text'])

# Convert DTM to DataFrame
dtm_df = pd.DataFrame(dtm.toarray(), columns=vectorizer.get_feature_names_out(), index=data['Title'])
dtm_df.head()

,00,00 04,00 04 20042005,00 04 20042005min,00 04 stock,00 cent,00 cent 00,00 cent august,00 indicated,00 indicated result,...,ﬁts ai,ﬁts ai addiɵon,ﬁts search,ﬁts search security,ﬁx,ﬁx foundaɵons,ﬁx foundaɵons economy,ﬂows,ﬂows eﬃcacy,ﬂows eﬃcacy global
Title,,,,,,,,,,,,,,,,,,,,,
"IMFC Statement by Christine Lagarde, President of the ECB",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"IMFC Statement by HE Haitham Al Ghais, Secretary General, OPEC",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"IMFC Statement by Ayman Al-Sayari, Governor of the Saudi Central Bank (SAMA)",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"IMFC Statement by Antoine Armand, Minister of the Economy, Finance and Industry, France",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"IMFC Statement by Luis Caputo, Minister of Economy, Argentina",1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
def read_file_to_list(file_path):
    """
    Reads a text file and returns a list of lines with leading and trailing whitespace removed.

    :param file_path: Path to the text file
    :return: List of stripped lines
    """
    try:
        with open(file_path, 'r') as file:
            content = file.read()
            lines = [line.strip().replace('_',' ').lower() for line in content.splitlines()]
        return lines
    except FileNotFoundError:
        print(f"Error: The file '{file_path}' was not found.")
        return []
    except Exception as e:
        print(f"An error occurred: {e}")
        return []

In [ ]:
env=

In [27]:
# Step 4: Manual Mapping to Topics
manual_mappings = {
    "Climate Change": ["climate", "carbon tax", "sustainability"],
    "Economic Growth": ["growth", "debt reform", "financial regulations"],
    "Environment":read_file_to_list(r'csvs\Environment_CSR.txt'),
    "Human Rights":read_file_to_list(r'csvs\HumanRights_CSR.txt'),

}

In [28]:
# Step 3: Count Topic Appearances in Each Document
def count_topic_appearances(text_tokens, lexicon):
    topic_counts = {topic: sum(text_tokens.count(term) for term in terms) for topic, terms in lexicon.items()}
    return topic_counts

data['Topic_Counts'] = data['Processed_Text'].apply(lambda x: count_topic_appearances(x, manual_mappings))

# Convert dictionary column into separate columns
topic_counts_df = pd.DataFrame(data['Topic_Counts'].tolist(), index=data.index)

# Step 4: Compute Total Topic Count Per Document
data['Total_Topic_Count'] = topic_counts_df.sum(axis=1)

# Step 5: Compute Topic Distribution (Normalized Frequency)
topic_distribution = topic_counts_df.div(data['Total_Topic_Count'], axis=0).fillna(0)

# Step 6: Merge Topic Distribution with Original Data
data = pd.concat([data, topic_distribution], axis=1)




In [29]:
data[['Title', 'Year'] + list(topic_distribution.columns)]

,Title,Year,Climate Change,Economic Growth,Environment,Human Rights
0,"IMFC Statement by Christine Lagarde, President...",2024,0.067568,0.162162,0.581081,0.189189
1,"IMFC Statement by HE Haitham Al Ghais, Secreta...",2024,0.000000,0.526316,0.368421,0.105263
2,"IMFC Statement by Ayman Al-Sayari, Governor of...",2024,0.019608,0.117647,0.441176,0.421569
3,"IMFC Statement by Antoine Armand, Minister of ...",2024,0.058824,0.011765,0.647059,0.282353
4,"IMFC Statement by Luis Caputo, Minister of Eco...",2024,0.041985,0.118321,0.564885,0.274809
...,...,...,...,...,...,...
559,IMFC Statement by the Honorable Domenico Sinis...,2004,0.043478,0.180124,0.503106,0.273292
560,"IMFC Statement by the Honorable John W. Snow, ...",2004,0.026316,0.157895,0.644737,0.171053
561,IMFC Statement by H.E. Sadakazu Tanigaki Minis...,2004,0.021739,0.043478,0.717391,0.217391
562,"IMFC Statement By James D. Wolfensohn, Preside...",2004,0.041322,0.148760,0.561983,0.247934


In [12]:
# Step 5: Alternative LDA Topic Modeling using Sklearn
lda = LatentDirichletAllocation(n_components=5, random_state=42)
lda_topics = lda.fit_transform(dtm)

# Function to display top words per topic
def display_topics(model, feature_names, num_words=5):
    for topic_idx, topic in enumerate(model.components_):
        top_words = [feature_names[i] for i in topic.argsort()[:-num_words - 1:-1]]
        print(f"Topic {topic_idx+1}: {', '.join(top_words)}")

print("\nLDA Topics :")
display_topics(lda, vectorizer.get_feature_names_out(), num_words=5)


LDA Topics :
Topic 1: growth, financial, imf, global, countries
Topic 2: growth, countries, financial, global, fund
Topic 3: financial, imf, growth, global, countries
Topic 4: financial, countries, fund, growth, global
Topic 5: financial, countries, global, growth, imf
